# TinyML - Relevance Vector Machine (Classifier)

In [32]:
#!pip install micromlgen
#!pip install sefr

## 1. Importing libraries

In [33]:
from micromlgen import port
import pandas as pd
import plotly.graph_objects as go
from sefr import SEFR
from sklearn.calibration import LabelEncoder
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn import metrics

## 2. Load Dataset

The Iris dataset is a classic dataset in the field of machine learning and statistics. It was introduced by Sir Ronald A. Fisher in 1936 as an example of discriminant analysis. The dataset is often used for educational purposes and is a common starting point for the practice of pattern classification.


Attributes:

- Sepal length (in centimeters)

- Sepal width (in centimeters)

- Petal length (in centimeters)


Species:

- 0 - Setosa

- 1 - Versicolor

In [34]:
# Load iris dataset
data = load_iris()

# Create a DataFrame
df_iris = pd.DataFrame(data.data, columns=data.feature_names)


# Add target variable to the DataFrame
df_iris['target'] = data.target

# Remove NaN values
df = df_iris.dropna(axis='rows') #remove NaN

# Display the DataFrame
print(df_iris.head())

   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

   target  
0       0  
1       0  
2       0  
3       0  
4       0  


In [35]:
df=df_iris.iloc[:100,0:4]

In [36]:
X=df.to_numpy()

# Converting string value to int type for labels: Setosa = 0, Versicolor = 1
y=df_iris.iloc[:100,-1]
y = LabelEncoder().fit_transform(y)

In [37]:
print(df.head())

   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                5.1               3.5                1.4               0.2
1                4.9               3.0                1.4               0.2
2                4.7               3.2                1.3               0.2
3                4.6               3.1                1.5               0.2
4                5.0               3.6                1.4               0.2


## 3. Dataset Visualization 

In [38]:
fig = go.Figure()


fig.add_trace(go.Scatter3d(x=df['sepal width (cm)'], y= df['petal length (cm)'], z=df['petal width (cm)'], mode='markers', marker=dict(color='blue')))

fig.update_layout(scene=dict(xaxis_title='Sepal Length (cm)', yaxis_title='Sepal Width (cm)', zaxis_title='Petal Width (cm)'),
                  scene_camera=dict(eye=dict(x=1.87, y=0.88, z=-0.64)),
                  width=1000, height=600)
fig.show()

In [39]:
print('Input shape: ', X.shape)
print('Target variable shape: ', y.shape)

Input shape:  (100, 4)
Target variable shape:  (100,)


## 4. Split into training and test data

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

## 5. Create the classification model

In [41]:
model = SEFR()

## 6. Train the model

In [42]:
model.fit(X_train, y_train)

## 7. Evaluating the model with the training data

In [43]:
training_predict = model.predict(X_train)

In [44]:
print(metrics.classification_report(y_train, training_predict, digits = 3))

              precision    recall  f1-score   support

           0      1.000     1.000     1.000        33
           1      1.000     1.000     1.000        37

    accuracy                          1.000        70
   macro avg      1.000     1.000     1.000        70
weighted avg      1.000     1.000     1.000        70



In [45]:
print(metrics.confusion_matrix(y_train, training_predict))

[[33  0]
 [ 0 37]]


In [46]:
print(f'Model accuracy: {round(metrics.accuracy_score(y_train, training_predict)*100,2)}%')

Model accuracy: 100.0%


## 8. Evaluating the model with test data

In [47]:
test_predict = model.predict(X_test)

In [48]:
print(metrics.classification_report(y_test, test_predict, digits = 3))

              precision    recall  f1-score   support

           0      1.000     1.000     1.000        17
           1      1.000     1.000     1.000        13

    accuracy                          1.000        30
   macro avg      1.000     1.000     1.000        30
weighted avg      1.000     1.000     1.000        30



In [49]:
print(metrics.confusion_matrix(y_test, test_predict))

[[17  0]
 [ 0 13]]


In [50]:
print(f'Model accuracy: {round(metrics.accuracy_score(y_test, test_predict)*100,2)}%')

Model accuracy: 100.0%


AttributeError: 'SEFR' object has no attribute 'decision_function'

## 9. Obtaining the model to be implemented in the microcontroller

In [51]:
print(port(model))

#pragma once
#include <cstdarg>
namespace Eloquent {
    namespace ML {
        namespace Port {
            class SEFR {
                public:
                    /**
                    * Predict class for features vector
                    */
                    int predict(float *x) {
                        return dot(x,   0.089210262828  , -0.097438891661  , 0.48245740461  , 0.68784598788 ) <= 2.0193337291074043 ? 0 : 1;
                    }

                protected:
                    /**
                    * Compute dot product
                    */
                    float dot(float *x, ...) {
                        va_list w;
                        va_start(w, 4);
                        float dot = 0.0;

                        for (uint16_t i = 0; i < 4; i++) {
                            const float wi = va_arg(w, double);
                            dot += x[i] * wi;
                        }

                        return dot;
                    }
         

## 10. Saves the template in a .h file

In [52]:
with open('./SEFRClassifier/SEFRClassifier.h', 'w') as file:
    file.write(port(model))